In [15]:
import sys
import os
import numpy as np
import cv2 
import albumentations as aug
import pandas as pd
from random import randint
import random
import matplotlib.pyplot as plt

In [2]:
image_path = '/home/foreground_img'   # foreground images path
back_path = '/home/background_img'    # background images path

In [3]:
class augmentor():
    
    def __init__(self):
        
         self.aug={}
            
         self.aug['rc']=aug.augmentations.transforms.RandomResizedCrop(1080, 1080, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=1.0) 
    
    def background(self,image):

        augmented=self.aug["rc"](image=image)
        return augmented['image']


In [4]:
augment=augmentor()

In [5]:
def getpoints(pts,pos):
    (yp,xp)=pos        
    points=pts+np.array([xp,yp])
    return points    

In [6]:
def getpos(fore,back):
    f_size=fore.shape
    b_size=back.shape
    
    assert(b_size[0]>f_size[0] and b_size[1]>f_size[1]) 
    
    x1=f_size[0]
    x2=b_size[0]
        
    y1=f_size[1]
    y2=b_size[1]
        
    x=randint(0,(x2-x1))
    y=randint(0,(y2-y1))
       
    return (x,y)

In [7]:
def stack_image(imgb,img2,pos):
       
    posx,posy=(pos)
    img1=imgb.copy()
    (w,h,c)=img2.shape
       
    crp=img1[posx:posx+w,posy:posy+h,:]
    crp[np.where(img2>=0)]=img2[np.where(img2>=0)].ravel()
    img1[posx:posx+w,posy:posy+h,:]=crp
    
    return img1

In [8]:
def save_image(img,name): 
    path = '/home/synthetic data/data'  # new images save path
    cv2.imwrite(os.path.join(path ,name+'.jpg'), img)

In [9]:
def get_dataframe(img_count,pts,path):
        
    [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]=pts
    name='image_'+str(img_count)+'.jpg'
    dataframe = pd.DataFrame([[path+name,  x1, y1, x2, y2, x3, y3, x4, y4]])
    return dataframe

In [10]:
def crop_with_kp(image,kp):
       
    pts=np.array((kp), np.int32)
        
    minx=min(pts[:,0])
    maxx=max(pts[:,0])
    miny=min(pts[:,1])
    maxy=max(pts[:,1])
    
    y,x,_=image.shape
    xl=randint(0,minx)
    xr=randint(maxx,x-1)
    yu=randint(0,miny)
    yd=randint(maxy,y-1)
    new_img=image[yu:yd,xl:xr,:]
    pts=pts-np.array(([xl,yu]))
    
    return new_img,pts

In [11]:
def getPerspectiveTransform(image):
    
    strength = random.randint(30,60)
    
    (yf,xf,c)=image.shape
    pts_base=np.float32(([(0,0),(xf,0),(xf,yf),(0,yf)]))
    
    pts=np.random.rand(4, 2)*random.uniform(-strength,0)+pts_base

    pts=pts.astype(np.float32)

    M = cv2.getPerspectiveTransform(pts, pts_base)
    trans_img = cv2.warpPerspective(image, M, (int(xf*1.5),int(yf*1.5)),cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)

    pts=np.array((pts), np.int32)
    pts[:,0]-=min(pts[:,0])
    pts[:,1]-=min(pts[:,1])
  
    return trans_img,pts

In [12]:
def generate_image(fore_img,back_img,f_size_ran,b_size_ran,rot_prob=1,randomcrop=False):

    fore=os.path.join(image_path, fore_img)
    back=os.path.join(back_path, back_img)
     
    fore=cv2.imread(fore)
    back=cv2.imread(back) 
    
    xf=randint(f_size_ran[0],f_size_ran[1])
    yf=int(fore.shape[0]/fore.shape[1]*xf)
            
    x=randint(int(xf*2),b_size_ran[1])
    y=randint(int(yf*2),b_size_ran[1])
        
    f_size=(xf,yf)
    b_size=(x,y)
    
    if(randomcrop):
        back=augment.background(back)

            
    fore=cv2.resize(fore,f_size,interpolation = cv2.INTER_AREA)
    back=cv2.resize(back,b_size,interpolation = cv2.INTER_AREA)        
            
    (yf,xf,c)=fore.shape
    pts=np.array(([(0,0),(xf,0),(xf,yf),(0,yf)]))# initial points
            
            
    if(randint(0,rot_prob)==0):
        fore=fore.astype("float32")
        fore=fore+1
        fore,pts=getPerspectiveTransform(fore)
        fore=fore-1
    
    pos=getpos(fore,back)
    new_img=stack_image(back,fore,pos)              
    point=getpoints(pts, pos)
                                                              
    img,pts=crop_with_kp(new_img,point)
            
    return img,pts

In [13]:
def generate(f_size_ran, b_size_ran,rot_prob=1,randomcrop=False):
    foreimage=os.listdir(image_path)
    backimage=os.listdir(back_path)
    dataframe=pd.DataFrame()
    misimg=0
    img_count=0
    total=0

    for bimg in backimage:

        for fimg in foreimage : 

            try:
                   
                img,pts=generate_image(fimg,bimg,f_size_ran,b_size_ran,rot_prob=rot_prob,randomcrop=randomcrop)
                img_count+=1
                path ='/home/synthetic data/data'   # new images save path
                df=get_dataframe(img_count,pts,path)
                    
                dataframe=dataframe.append(df,ignore_index=True)
                img_name='image_'+str(img_count)
                save_image(img,img_name)
                   
            except :
                misimg+=1

            finally:
                total+=1
          
            
    dataframe.to_csv('synthetic_data.csv', header=['image_path', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
    print("processed ",total,"samples\nmissed  ",misimg,"  samples")    






In [14]:
generate((300,390),(1000,1024),randomcrop=True)

processed  1060 samples
missed   279   samples
